# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

from dotenv import load_dotenv
from pydantic import BaseModel
import datasets
from datasets import load_dataset

from loguru import logger
import pandas as pd

datasets.logging.set_verbosity_error()

sys.path.insert(0, '..')

load_dotenv()

from src.utils.data_prep import parse_dt, handle_dtypes
from src.visualization.setup import color_scheme, fsds_colors

# Controller

In [3]:
class Args(BaseModel):
    hf_dataset_path: str = "McAuley-Lab/Amazon-Reviews-2023"
    report_sample_num_rows: int = 10000
    random_seed: int = 41

args = Args()

# Load data

In [4]:
dataset = load_dataset(args.hf_dataset_path, name="0core_timestamp_Video_Games", trust_remote_code=True)
# To load all user review data such as review_text, helpful score, use: name="raw_review_All_Beauty"

In [5]:
train_raw_df = dataset['train'].to_pandas()

In [6]:
train_df = (
    train_raw_df
    .pipe(parse_dt)
    .pipe(handle_dtypes)
)

In [7]:
train_df

,user_id,parent_asin,rating,timestamp
0,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B07SRWRH5D,5.0,2020-04-16 15:31:54.941
1,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B07DK1H3H5,4.0,2020-12-17 06:33:24.795
2,AGXVBIUFLFGMVLATYXHJYL4A5Q7Q,B07MFMFW34,5.0,2017-03-30 12:37:11.000
3,AFTC6ZR5IKNRDG5JCPVNVMU3XV2Q,B00HUWA45W,5.0,2015-03-29 01:18:52.000
4,AFTC6ZR5IKNRDG5JCPVNVMU3XV2Q,B0BCHWZX95,5.0,2019-12-29 16:40:34.017
...,...,...,...,...
3847036,AFFYUOGDNI3JXGA22RSGHT2FJZ5A,B0001AO01Y,1.0,2004-03-31 04:20:36.000
3847037,AEBGL7K4NBN2ALC6O353GJ3BIBYQ,B074QM6F7X,1.0,2018-07-12 22:12:22.360
3847038,AHTYCU6NSHJ4BY7R2YNDIDGUXS6Q,B089F1BD4W,5.0,2020-09-23 12:24:51.023
3847039,AF4KDWDGCJSUEB7JESQZ6ZWD2LNA,B0BN942894,5.0,2020-03-22 14:39:59.168


# Insert data into Postgre

In [8]:
from sqlalchemy import create_engine

In [9]:
# PostgreSQL connection details
username = os.environ.get("POSTGRES_USER")
password = os.environ.get("POSTGRES_PASSWORD")
host = 'localhost'
port = '5432'
database = os.environ.get("POSTGRES_DB")

# Create a connection string and engine outside the function
connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

# Table name where data will be inserted
table_name = 'amz_review_rating_raw'

In [10]:
# Insert data from start_date to end_date - 1 day
start_date = '2020-08-20'
end_date = '2020-08-27'

to_insert_df = train_df.loc[lambda df: df['timestamp'].between(start_date, end_date)]

In [11]:
to_insert_df

,user_id,parent_asin,rating,timestamp
383,AGLQQCIS5V6EGUCS5SSNHWZHQM6Q,B09WWDTXPB,4.0,2020-08-26 14:50:40.694
766,AFTC72W2CBE3CDH3ATFRDL4RQSAA,B086JQGB7W,2.0,2020-08-22 00:53:10.879
1007,AE3OUW2QZVYKTDJ33WRUNO75WIXA,B09XB745H6,3.0,2020-08-24 17:35:58.570
1226,AHIUDIC4IOC3TUWC25ISKZZ4W6JA,B079Y44LDC,5.0,2020-08-24 21:18:30.127
1227,AHIUDIC4IOC3TUWC25ISKZZ4W6JA,B07HGN2FMY,5.0,2020-08-24 21:18:48.779
...,...,...,...,...
3845664,AEOYVVS4BFOXS5ISB6PMTYOMMYWA,B06XH1NTFY,2.0,2020-08-22 03:25:46.060
3845821,AERAKQ5BPLF7IVYIVAOMDFMN45WQ,B09TH98R9G,5.0,2020-08-23 23:46:53.569
3846231,AFXQTNIDEUOD4HNCHVH5H4GI76WQ,B081HYC7MY,5.0,2020-08-20 20:19:02.195
3846502,AGANXL2J7WZCXFOLDRTBVEIYWWJA,B07DRQVH6K,4.0,2020-08-25 03:43:00.817


In [12]:
to_insert_df.to_sql(table_name, engine, if_exists='append', index=False)

588

# Archive

#### Insert a new batch of a new day to verify materialize incremental